## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'HourlyRate',
             'JobInvolvement', 'JobSatisfaction', 'NumCompaniesWorked', 'OverTime', 'StockOptionLevel']
# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes

Age                         int64
DistanceFromHome            int64
Education                   int64
EnvironmentSatisfaction     int64
HourlyRate                  int64
JobInvolvement              int64
JobSatisfaction             int64
NumCompaniesWorked          int64
OverTime                   object
StockOptionLevel            int64
dtype: object

In [10]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_df["OverTime"] = X_df["OverTime"].apply(lambda x: 1 if x == "Yes" else 0)

C:\Users\brown\AppData\Local\Temp\ipykernel_39800\1767932470.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df["OverTime"] = X_df["OverTime"].apply(lambda x: 1 if x == "Yes" else 0)


In [11]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [12]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)

In [14]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
OHE_dept = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
OHE_dept.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept_encoded = OHE_dept.transform(y_train[['Department']])
y_test_dept_encoded = OHE_dept.transform(y_test[['Department']])

In [16]:
# Create a OneHotEncoder for the Attrition column
OHE_attrition = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
OHE_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr_encoded = OHE_attrition.transform(y_train[['Attrition']])
y_test_attr_encoded = OHE_attrition.transform(y_test[['Attrition']])

## Part 2: Create, Compile, and Train the Model

In [17]:
# Find the number of columns in the X training data.
input_shape = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_shape,))

# Create at least two shared layers
hidden_layer1 = layers.Dense(64, activation='relu')(input_layer)
hidden_layer2 = layers.Dense(32, activation='relu')(hidden_layer1)

In [18]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = layers.Dense(16, activation='relu')(hidden_layer2)

# Create the output layer
dept_output = layers.Dense(y_train_dept_encoded.shape[1], activation='softmax', name='dept_output')(dept_hidden)


In [19]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_hidden = layers.Dense(16, activation='relu')(hidden_layer2)

# Create the output layer
attr_output = layers.Dense(y_train_attr_encoded.shape[1], activation='sigmoid', name='attr_output')(attr_hidden)


In [20]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attr_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'dept_output': 'categorical_crossentropy', 'attr_output': 'binary_crossentropy'},
              metrics={'dept_output': 'accuracy', 'attr_output': 'accuracy'})


# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        704 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output (Dense) │ (None, 3)         │         51 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attr_output (Dense) │ (None, 2)         │         34 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,925 (15.33 KB)

 Trainable params: 3,925 (15.33 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Train the model
history = model.fit(X_train_scaled, {'dept_output': y_train_dept_encoded, 'attr_output': y_train_attr_encoded},
                    validation_data=(X_test_scaled, {'dept_output': y_test_dept_encoded, 'attr_output': y_test_attr_encoded}),
                    epochs=50, batch_size=32)

Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - attr_output_accuracy: 0.7467 - attr_output_loss: 0.6441 - dept_output_accuracy: 0.4227 - dept_output_loss: 1.0542 - loss: 1.6985 - val_attr_output_accuracy: 0.8673 - val_attr_output_loss: 0.4564 - val_dept_output_accuracy: 0.6667 - val_dept_output_loss: 0.8283 - val_loss: 1.2918
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attr_output_accuracy: 0.8235 - attr_output_loss: 0.4889 - dept_output_accuracy: 0.6236 - dept_output_loss: 0.8298 - loss: 1.3188 - val_attr_output_accuracy: 0.8673 - val_attr_output_loss: 0.3751 - val_dept_output_accuracy: 0.6667 - val_dept_output_loss: 0.7719 - val_loss: 1.1659
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attr_output_accuracy: 0.8219 - attr_output_loss: 0.4628 - dept_output_accuracy: 0.6650 - dept_output_loss: 0.7516 - loss: 1.2144 - val_attr_output_accuracy: 0.8673 - val_attr_output_loss: 0.3754 - val_dept_output_accuracy: 0.6667 - val_dept_output_loss: 0.7722 - val_loss: 1.1649
Epo

In [22]:
# Evaluate the model with the testing data
test_loss, dept_loss, attr_loss, dept_acc, attr_acc = model.evaluate(X_test_scaled, {'dept_output': y_test_dept_encoded, 'attr_output': y_test_attr_encoded})


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attr_output_accuracy: 0.8491 - attr_output_loss: 0.4080 - dept_output_accuracy: 0.5874 - dept_output_loss: 0.9344 - loss: 1.3488 


In [ ]:
# Print the accuracy for both department and attrition
print(f"Department Prediction Accuracy: {dept_acc:.4f}")
print(f"Attrition Prediction Accuracy: {attr_acc:.4f}")

Department Prediction Accuracy: 0.8503
Attrition Prediction Accuracy: 0.6054


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is useful but not always the best metric, especially if the dataset is imbalanced. For attrition, precision, recall, and F1-score provide better insights since false positives and false negatives matter. For department prediction, accuracy is more reliable, but a confusion matrix or top-k accuracy could add more value.
2. For the attrition output, I used sigmoid since it is a binary classification problem, which outputs a probability between 0 and 1. For the department output, I used softmax because it is a multi-class classification problem, ensuring the outputs sum to 1 and represent class probabilities.
3. The model could be improved by tuning hyperparameters, such as the number of layers, neurons, and learning rate. Using techniques like dropout or batch normalization could help prevent overfitting. Additionally, handling class imbalance in attrition prediction through SMOTE or class weighting might improve performance.